In [12]:
import os
from glob import glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras import layers

num_epochs = 10
batch_size = 100
learning_rate = 0.001
dropout_rate = 0.25
input_shape = (480,640,3)
num_classes = 10

#input shape setting
inputs = layers.Input(shape=input_shape)

# Feature Extraction
net = layers.Conv2D(32, 3, 1, padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, 3, 1, padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPool2D((2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, 3, 1, padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, 3, 1, padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPool2D((2,2))(net)
net = layers.Dropout(dropout_rate)(net)

## Fully connected layer
net = layers.Conv2D(128, 3, 1, padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='TestCNN')

#model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # (주의) Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

train_paths = glob('noise2020')[0:10000]
test_paths = glob('train2020')[10000:11000]

def get_label(path):
    return path.split('\\')[-1].replace('.jpg', '')

train_label_names = [get_label(path) for path in train_paths]

class_names = [get_label(path) for path in train_paths]
class_names = np.unique(class_names)

def get_label_onehot_encoding(path):
    fname = tf.strings.split(path, '\\')[-1]
    cls_name = tf.strings.regex_replace(fname, '.jpg', '')
    tf_onehot_encoding = tf.cast(class_names==cls_name, tf.uint8)
    return tf_onehot_encoding

def read_image_label(path, label):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32)/255.
    label = get_label_onehot_encoding(path)
    return image, label

test_label_names = [get_label(path) for path in test_paths]

def image_preprocess(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image, label

In [13]:
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_label_names))
train_dataset = train_dataset.map(read_image_label)
train_dataset = train_dataset.map(image_preprocess)
train_dataset = train_dataset.batch(batch_size)
#dataset = dataset.shuffle(buffer_size=len(train_paths)) 
train_dataset = train_dataset.repeat()

batch_size = 32
test_dataset = tf.data.Dataset.from_tensor_slices((test_paths, test_label_names))
test_dataset = test_dataset.map(read_image_label)
test_dataset = test_dataset.batch(batch_size)
#dataset = dataset.shuffle(buffer_size=len(test_dataset)) 
test_dataset = test_dataset.repeat()

train_steps_per_epoch = len(train_paths) // batch_size
validataion_steps_per_epoch = len(test_paths) // batch_size

model.fit_generator(
    train_dataset, 
    steps_per_epoch=train_steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validataion_steps_per_epoch, 
    epochs=num_epochs
)


TypeError: in user code:

    <ipython-input-6-f047c2562a35>:65 read_image_label  *
        gfile = tf.io.read_file(path)
    C:\Users\USER\anaconda3\lib\site-packages\tensorflow\python\ops\gen_io_ops.py:574 read_file  **
        _, _, _op, _outputs = _op_def_library._apply_op_helper(
    C:\Users\USER\anaconda3\lib\site-packages\tensorflow\python\framework\op_def_library.py:492 _apply_op_helper
        raise TypeError("%s expected type of %s." %

    TypeError: Input 'filename' of 'ReadFile' Op has type float32 that does not match expected type of string.
